# Establish a database connection and retrieve the inventory and User settings

Internally, geoslurp uses the python sqlalchemy tools to establish a connection with the database. The information provided is therefore similar (hostname+port, username, password). This information can be provided directly to the constructor of [GeoslurpConnector](../docs/reference/geoslurp.db.html#geoslurp.db.connector.GeoslurpConnector). Alternatively, the convenience function [geoslurpConnect](../docs/reference/geoslurp.db.html#geoslurp.db.geoslurpdb.geoslurpConnect)  uses [local settings](../docs/installation.html#local-configuration-settings)  from the file ``.geoslurp_lastused.yaml`` in the users's home. This has the advantage that possible sensitive information (logins, passwords) does not need to be stored in notebooks such as these.

## Read-only user versus priviliged user
For the majority of the query operations, it is not needed to have write privileges to the database tables. For this purpose, it is recommended to access the database with a readonly user to avoid making unwanted changes to the tables. How this is done is shown below.

## Using geoslurpConnect
There are several ways to specify passwords, some which are more secure than others.
1. **Use a keyring recognized by python keyring**. This requires that [python keyring  is installed with a suitable backend](https://pypi.org/project/keyring/). When the option ``useKeyring=true`` is specified in ``${HOME}/.geoslurp_lastused.yaml``, users will be prompted for a  password when they connect for the first time, but subsequent accesses don't require entering a password, and are therefore non-interactive.
2. **Use an environment variable**. If ``useKeyRing=false``, one can use an environment variable to specify the password e.g.: ``export GEOSLURP_PGPASS="supersecretpassword``, or ``export GEOSLURP_PGPASSRO`` (for the readonly user).
3. If both methods 1 and 2 fail. one will be interactively prompted for a password

In [1]:
#Connect to a database using the settings in ${HOME}/.geoslurp_lastused.yaml
from geoslurp.db import geoslurpConnect

#connect to the database with the normal user as specified in ${HOME}/.geoslurp_lastused.yaml
dbcon=geoslurpConnect(readonlyuser=False)

#Connect to a database with the readonly user as specified in ${HOME}/.geoslurp_lastused.yaml
dbconReadOnly=geoslurpConnect()




## Alternatively one can use the class [GeoslurpConnector](../docs/reference/geoslurp.db.html#geoslurp.db.connector.GeoslurpConnector) and provide all connections details manually
This will **not** consult the file `${HOME}/.geoslurp_lastused.yaml` and can look like:

`from geoslurp.db import GeoslurpConnector`

Interactively (i.e. prompt for a password):

`dbcon=GeoslurpConnector(host="geoslurphost",user="yourusername")`

Non-interactively (**not recommended** for use in code which may be shared, such as jupyter notebooks):
`dbcon=GeoslurpConnector(host="geoslurphost",user="slurpy",passwd="supersecretpassword")`

## Loading the Dataset Inventory
The [Inventory](../docs/reference/geoslurp.db.html#geoslurp.db.inventory.Inventory) class is coupled to the *admin.inventory* table in the database, and contains information on registered datasets. One can load the data the table by initializing a Inventory class instance with a database connection such as obtained from above. Entries (i.e.) can be requested or iterated over as shown below.

In [5]:
from geoslurp.db import Inventory


inventory=Inventory(dbcon)

entry=inventory["globalgis.gshhs_c"]
print(entry.scheme, entry.dataset, entry.pgfunc,entry.owner,entry.lastupdate,entry.version,entry.cache,entry.datadir,entry.data)


for entry in inventory:
    print(entry.scheme, entry.dataset, entry.owner)



globalgis gshhs_c None roelof 2019-02-27 09:47:02.441119 (0, 0, 0) None None {'GSHHGversion': [2, 3, 7]}
altim rads_j1_a roelof
OceanObs psmsl_rlr_monthly roelof
OceanObs psmsl_rlr_annual roelof
OceanObs psmsl_met_monthly roelof
globalgis wdbii_river_l roelof
globalgis wdbii_river_i roelof
Gravity itsg_grace2018_monthly_n40 roelof
globalgis wdbii_river_h roelof
globalgis gshhs_c roelof
globalgis gshhs_l roelof
globalgis wdbii_river_f roelof
globalgis gshhs_i roelof
globalgis gshhs_h roelof
globalgis wdbii_border_c roelof
globalgis gshhs_f roelof
globalgis wdbii_river_c roelof
globalgis wdbii_border_l roelof
Gravity itsg_grace2018_daily_background roelof
globalgis wdbii_border_i roelof
Gravity itsg_grace2018_monthly_background roelof
globalgis wdbii_border_h roelof
Gravity itsg_grace2018_monthly_n60 roelof
globalgis wdbii_border_f roelof
globalgis ne_110m_coastline roelof
Gravity itsg_grace2018_monthly_n96 roelof
cryo 00_rgi60_o2regions roelof
Gravity gracel2_csr_rl06 roelof
Gravity gra

## Setting and getting usersettings from the database

In contrast to settings which are specific for a host and user, the database also contains a table, *admin.settings* which contains settings which are specific to the server and user. These can be loaded by initializing a [Settings](../docs/reference/geoslurp.db.html#geoslurp.db.settings.Settings) class. Note that the read-only user does not have any settings stored in the database and has no read permission to the table.

### Default settings
The *admin.settings* table has a *default* entry which contains settings which are valid for all users, unless they are overruled by the user themselves.

### Storing Authentication details
Users may need to access webservices which require authentication details. These authentication details are also stored in the settings table of the database, but as this is sensitive information the authentication data is encrypted in the database. The encryption is based on the user's password, **which is unknown to the database** (only the hash is known to the database). Consequently, the authentication details can only be decrypted on clients. The implication of this is that **it is not possible to retrieve the authentication data, when a user forgets their password**.

In [4]:
from geoslurp.db import Settings
from random import random
conf=Settings(dbcon)

#to show the user settings:
#conf.show()

# try setting an entry
testvalue=random()
conf["test"]=testvalue
#synchronize the settings with the database
conf.update()

#load the settings from the database
conf2=Settings(dbcon)
if testvalue == conf2["test"]:
    print("succesfully set and synchronize an user setting")

#remove entry
del conf["test"]



#store new authentication details



succesfully set and synchronize an user setting
